## Get waze alerts history and open events from bigquery database

In [1]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


In [3]:
# modules
import requests, json, time, pandas as pd, numpy as np, urllib
from datetime import datetime as dt
from matplotlib.path import Path as mpl_path
from google.cloud import bigquery
from google.oauth2 import service_account

# google cloud auth
# project_id = 'pluvia-360323'
google_credentials = 'credentials/pluvia-360323-eae2907a9c98.json'
google_credentials = service_account.Credentials.from_service_account_file(google_credentials)
client = bigquery.Client(credentials=google_credentials)

fields_bq = [
    'id_pop', 'id_evento', 'bairro', 'data_inicio', 'data_fim',
    'prazo', 'descricao', 'gravidade', 'latitude', 'longitude',
    'status', 'tipo', 'data_particao', 'pop_titulo'
]

C:\Users\luisr\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
def bigquery_request(query):
    query_job = client.query(query)
    return pd.DataFrame(list(map(dict, query_job.result())))

def where_clause(items):
    return ' AND '.join([' '.join([key, 'IN', f"""({', '.join([f'"{value}"' for value in (values if type(values) is list else values.split(','))])})"""]) for key, values in items.items()])

def get_waze_bq_history(
    start=dt(2023,1,1), end='now',
    where={}, select='*', as_datetime=False,
):
    waze_query = '''
    SELECT {} FROM `datario.adm_cor_comando.ocorrencias` WHERE data_particao BETWEEN "{}" AND "{}" AND data_inicio BETWEEN "{}" AND "{}"
    '''
    if end == 'now': end = dt.now()
    if type(select) is list: select = ', '.join(select)
    query = waze_query.format(select, str(start.date()), str(end.date()), str(start), str(end))
    if len(where):
        query = ' '.join([query, 'AND', where_clause(where)])
    history = bigquery_request(query)
    if len(history):
        if not as_datetime:
            if 'data_particao' in history.columns: history['data_particao'] = str(history['data_particao'])
            for col in ['data_inicio', 'data_fim']:
                if col in history.columns:
                    history[col] = history[col].map(lambda stamp: stamp.strftime('%Y-%m-%d %X'))
        return history
    else:
        return pd.DataFrame(columns=fields_bq)

def get_waze_bq_open_events(
    at=dt.now(), where={}, select='*',
    between=7*pd.offsets.Day(), as_datetime=False,
):
    comando_query = '''
        SELECT {} FROM `datario.adm_cor_comando.ocorrencias`
        WHERE data_particao BETWEEN "{}" AND "{}"
        AND data_inicio <= "{}" AND data_fim >= "{}"
    '''
    start, end = at - between / 2, at + between / 2
    query = comando_query.format(select, str(start.date()), str(end.date()), str(at), str(at))
    if len(where): query = ' '.join([query, 'AND', where_clause(where)])
    history = bigquery_request(query)
    if len(history):
        history['pop_titulo'] = history['id_pop'].astype(int).map(pops_map) # include event pop title
        if not as_datetime:
            if 'data_particao' in history.columns: history['data_particao'] = str(history['data_particao'])
            for col in ['data_inicio', 'data_fim']:
                if col in history.columns:
                    history[col] = history[col].map(lambda stamp: stamp.strftime('%Y-%m-%d %X'))
        return history
    else:
        return pd.DataFrame(columns=fields_bq)